<a href="https://colab.research.google.com/github/mlccgkpol/makeMore/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import torch
%matplotlib inline

In [ ]:
words = open("names.txt",'r').read().splitlines()

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [ ]:
N = torch.zeros((27,27), dtype = torch.int32)
for w in words:
  chs = ['.'] + list(w) + ['.']
  # print(chs,"chs")
  for ch1, ch2 in zip(chs, chs[1:]):
    N[stoi[ch1],stoi[ch2]]+=1
    # print(ch1,ch2)

In [ ]:
def buildDiagram(N):
  plt.figure(figsize=(16,16))
  plt.imshow(N, cmap = 'Blues')
  for i in range(27):
    for j in range(27):
      chstr = itos[i] + itos[j]
      plt.text(j,i,chstr, ha = "center", va = "bottom", color = 'gray')
      plt.text(j,i,N[i,j].item(), ha = "center", va = "top", color = 'gray')
  plt.axis("off");

In [ ]:
P = (N + 100).float() #model smoothing and N+100 adds 100 to each element.
print(P.shape)
# P.sum(1,keepdim=True)
P /= P.sum(1, keepdim=True)
# 0 here means on 0th dimension, it will become 1 and it will sum across the column.
# is it possible to divide [27,27] an [27,1]?? good point.

# Broadcasting rules in pytorch ->

# Each tensor has at least one dimension.
# When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.


In [ ]:
def generateStrings(P):
  q = torch.Generator().manual_seed(2147483647)
  ix = 0
  for i in range(5):
    out = []
    while True:
      p = P[ix]
      ix = torch.multinomial(p,num_samples=1,replacement=True,generator=q).item()
      if ix == 0:
        break;
      out.append(itos[ix])
    print(''.join(out))


In [ ]:
# 27,27
# 27

#27,27
# 1,27 it will shift the 27 right.

# so we will have problem instead of normalizing rows, we are normalizing column.

# Thats why (keepdim = true) to get desired result when required.

In [ ]:
for w in ["rohit"]:
  chs = ['.'] + list(w) + ['.']
  logProb = 0;
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logProb += -torch.log(prob)
logProb/=3
print(f'{ch1}{ch2} -> {prob:.4f} -> {logProb: .4f}')

In [ ]:
# minimize negative log likelihood.

In [196]:
# create the training set of bigrams (x,y)
xs,ys = [],[]

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  # print(chs,"chs")
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [197]:
xs

tensor([ 0,  5, 13, 13,  1])

In [ ]:
# there are two ways to create tensor
# torch.tensor
# torch.Tensor